In [49]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

In [50]:
df = pd.read_csv('US_Regional_Sales_Data.csv')

In [51]:
print(df.shape)

(7991, 16)


In [52]:
df.head()

,OrderNumber,Sales Channel,WarehouseCode,ProcuredDate,OrderDate,ShipDate,DeliveryDate,CurrencyCode,_SalesTeamID,_CustomerID,_StoreID,_ProductID,Order Quantity,Discount Applied,Unit Cost,Unit Price
0,SO - 000101,In-Store,WARE-UHY1004,31/12/17,31/5/18,14/6/18,19/6/18,USD,6,15,259,12,5,0.075,"1,001.18","1,963.10"
1,SO - 000102,Online,WARE-NMK1003,31/12/17,31/5/18,22/6/18,2/7/18,USD,14,20,196,27,3,0.075,"3,348.66","3,939.60"
2,SO - 000103,Distributor,WARE-UHY1004,31/12/17,31/5/18,21/6/18,1/7/18,USD,21,16,213,16,1,0.050,781.22,"1,775.50"
3,SO - 000104,Wholesale,WARE-NMK1003,31/12/17,31/5/18,2/6/18,7/6/18,USD,28,48,107,23,8,0.075,"1,464.69","2,324.90"
4,SO - 000105,Distributor,WARE-NMK1003,10/4/18,31/5/18,16/6/18,26/6/18,USD,22,49,111,26,8,0.100,"1,476.14","1,822.40"


### Data Summary

In [53]:
def summary(df):
    sum=pd.DataFrame(df.dtypes,columns=['Data Type'])
    sum["Missing"]=df.isnull().sum()
    sum['Unique values']=df.nunique().values
    return sum

In [54]:
summary(df)

,Data Type,Missing,Unique values
OrderNumber,object,0,7991
Sales Channel,object,0,4
WarehouseCode,object,0,6
ProcuredDate,object,0,11
OrderDate,object,0,945
ShipDate,object,0,966
DeliveryDate,object,0,966
CurrencyCode,object,0,1
_SalesTeamID,int64,0,28
_CustomerID,int64,0,50


In [55]:
df= df.drop(['OrderNumber', 'CurrencyCode'], axis=1)
df

,Sales Channel,WarehouseCode,ProcuredDate,OrderDate,ShipDate,DeliveryDate,_SalesTeamID,_CustomerID,_StoreID,_ProductID,Order Quantity,Discount Applied,Unit Cost,Unit Price
0,In-Store,WARE-UHY1004,31/12/17,31/5/18,14/6/18,19/6/18,6,15,259,12,5,0.075,"1,001.18","1,963.10"
1,Online,WARE-NMK1003,31/12/17,31/5/18,22/6/18,2/7/18,14,20,196,27,3,0.075,"3,348.66","3,939.60"
2,Distributor,WARE-UHY1004,31/12/17,31/5/18,21/6/18,1/7/18,21,16,213,16,1,0.050,781.22,"1,775.50"
3,Wholesale,WARE-NMK1003,31/12/17,31/5/18,2/6/18,7/6/18,28,48,107,23,8,0.075,"1,464.69","2,324.90"
4,Distributor,WARE-NMK1003,10/4/18,31/5/18,16/6/18,26/6/18,22,49,111,26,8,0.100,"1,476.14","1,822.40"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7986,In-Store,WARE-MKL1006,26/9/20,30/12/20,7/1/21,14/1/21,9,41,339,29,1,0.075,121.94,234.5
7987,Online,WARE-NMK1003,26/9/20,30/12/20,2/1/21,4/1/21,14,29,202,3,6,0.050,"1,921.56","3,202.60"
7988,Online,WARE-UHY1004,26/9/20,30/12/20,23/1/21,26/1/21,14,32,241,35,5,0.200,"2,792.76","3,825.70"
7989,Online,WARE-NMK1003,26/9/20,30/12/20,20/1/21,25/1/21,20,42,112,36,8,0.100,804,"1,072.00"


### Convert data types

In [56]:
df["Unit Cost"] = df["Unit Cost"].str.replace(",", "").astype(float)
df["Unit Price"] = df["Unit Price"].str.replace(",", "").astype(float)

In [58]:
date= df[['ProcuredDate', 'OrderDate', 'ShipDate', 'DeliveryDate']]

In [59]:
for i in date.columns:
    date[i]= pd.to_datetime(date[i])
date.head()

C:\Users\Ankita Mishra\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,ProcuredDate,OrderDate,ShipDate,DeliveryDate
0,2017-12-31,2018-05-31,2018-06-14,2018-06-19
1,2017-12-31,2018-05-31,2018-06-22,2018-02-07
2,2017-12-31,2018-05-31,2018-06-21,2018-01-07
3,2017-12-31,2018-05-31,2018-02-06,2018-07-06
4,2018-10-04,2018-05-31,2018-06-16,2018-06-26


In [60]:
delay=date['DeliveryDate']-date['OrderDate']
delay

0        19 days
1      -113 days
2      -144 days
3        36 days
4        26 days
          ...   
7986     15 days
7987     92 days
7988     27 days
7989     26 days
7990     20 days
Length: 7991, dtype: timedelta64[ns]

In [61]:
shelf_duration = date['OrderDate']-date['ProcuredDate']
shelf_duration

0       151 days
1       151 days
2       151 days
3       151 days
4      -126 days
          ...   
7986     95 days
7987     95 days
7988     95 days
7989     95 days
7990     95 days
Length: 7991, dtype: timedelta64[ns]

In [62]:
date['delivery_delay'] = pd.Series(delay)
date['shelf_duration'] = pd.Series(shelf_duration)
date.head()

C:\Users\Ankita Mishra\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
C:\Users\Ankita Mishra\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,ProcuredDate,OrderDate,ShipDate,DeliveryDate,delivery_delay,shelf_duration
0,2017-12-31,2018-05-31,2018-06-14,2018-06-19,19 days,151 days
1,2017-12-31,2018-05-31,2018-06-22,2018-02-07,-113 days,151 days
2,2017-12-31,2018-05-31,2018-06-21,2018-01-07,-144 days,151 days
3,2017-12-31,2018-05-31,2018-02-06,2018-07-06,36 days,151 days
4,2018-10-04,2018-05-31,2018-06-16,2018-06-26,26 days,-126 days


In [63]:
date['delivery_delay'] = date['delivery_delay'].apply(lambda x:int(str(x).split(' ')[0]))
date['shelf_duration'] = date['shelf_duration'].apply(lambda x:int(str(x).split(' ')[0]))
date.head()

C:\Users\Ankita Mishra\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
C:\Users\Ankita Mishra\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,ProcuredDate,OrderDate,ShipDate,DeliveryDate,delivery_delay,shelf_duration
0,2017-12-31,2018-05-31,2018-06-14,2018-06-19,19,151
1,2017-12-31,2018-05-31,2018-06-22,2018-02-07,-113,151
2,2017-12-31,2018-05-31,2018-06-21,2018-01-07,-144,151
3,2017-12-31,2018-05-31,2018-02-06,2018-07-06,36,151
4,2018-10-04,2018-05-31,2018-06-16,2018-06-26,26,-126
